https://github.com/slehkyi/notebooks-for-articles/blob/master/web_scrapping_understatcom_for_xG_dataset.ipynb

In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import requests
from bs4 import BeautifulSoup

In [6]:
# create urls for all seasons of all leagues
base_url = 'https://understat.com/league'
# leagues = ['EPL']
leagues = ['La_liga', 'EPL', 'Bundesliga', 'Serie_A', 'Ligue_1', 'RFPL']
seasons = ['2014', '2015', '2016', '2017', '2018']
# seasons = ['2014', '2015']

In [7]:
# Starting with latest data for Spanish league, because I'm a Barcelona fan
url = base_url+'/'+leagues[0]+'/'+seasons[1]
res = requests.get(url)
soup = BeautifulSoup(res.content, "lxml")

# Based on the structure of the webpage, I found that data is in the JSON variable, under <script> tags
scripts = soup.find_all('script')

# Check our <script> tags
for el in scripts:
    print('*'*50)
    print(el.text)

**************************************************

**************************************************

**************************************************

**************************************************

**************************************************

**************************************************

**************************************************

**************************************************

**************************************************

**************************************************

**************************************************

**************************************************

**************************************************

**************************************************

**************************************************

**************************************************

**************************************************

**************************************************

**************************************************

************

In [15]:
import json

string_with_json_obj = ''

# Find data for teams
for el in scripts:
    if 'teamsData' in str(el):
        string_with_json_obj = str(el).strip()

# print(string_with_json_obj)

# strip unnecessary symbols and get only JSON data
ind_start = string_with_json_obj.index("('")+2
ind_end = string_with_json_obj.index("')")
json_data = string_with_json_obj[ind_start:ind_end]

json_data = json_data.encode('utf8').decode('unicode_escape')

In [17]:
# convert JSON data into Python dictionary
data = json.loads(json_data)
# print(data)
print(data.keys())
print('='*50)
print(data['138'].keys())
print('='*50)
print(data['138']['id'])
print('='*50)
print(data['138']['title'])
print('='*50)
print(data['138']['history'][0])

# Print pretty JSON data to check out what we have there
s = json.dumps(data, indent=4, sort_keys=True)
print(s)

dict_keys(['137', '138', '139', '140', '141', '142', '143', '144', '145', '146', '147', '148', '149', '150', '151', '152', '153', '154', '155', '156'])
dict_keys(['id', 'title', 'history'])
138
Sevilla
{'h_a': 'a', 'xG': 0.856601, 'xGA': 0.98272, 'npxG': 0.856601, 'npxGA': 0.98272, 'ppda': {'att': 179, 'def': 20}, 'ppda_allowed': {'att': 146, 'def': 31}, 'deep': 3, 'deep_allowed': 6, 'scored': 0, 'missed': 0, 'xpts': 1.2439, 'result': 'd', 'date': '2015-08-21 22:30:00', 'wins': 0, 'draws': 1, 'loses': 0, 'pts': 1, 'npxGD': -0.1261190000000001}
{
    "137": {
        "history": [
            {
                "date": "2015-08-21 22:30:00",
                "deep": 6,
                "deep_allowed": 3,
                "draws": 1,
                "h_a": "h",
                "loses": 0,
                "missed": 0,
                "npxG": 0.98272,
                "npxGA": 0.856601,
                "npxGD": 0.1261190000000001,
                "ppda": {
                    "att": 146,
       

In [19]:
# Get teams and their relevant ids and put them into separate dictionary
teams = {}
for id in data.keys():
    teams[id] = data[id]['title']

In [20]:
# EDA to get a feeling of how the JSON is structured
# Column names are all the same, so we just use first element
columns = []
# Check the sample of values per each column
values = []
for id in data.keys():
    columns = list(data[id]['history'][0].keys())
    values = list(data[id]['history'][0].values())
    break

print(columns)
print(values)

['h_a', 'xG', 'xGA', 'npxG', 'npxGA', 'ppda', 'ppda_allowed', 'deep', 'deep_allowed', 'scored', 'missed', 'xpts', 'result', 'date', 'wins', 'draws', 'loses', 'pts', 'npxGD']
['h', 0.98272, 0.856601, 0.98272, 0.856601, {'att': 146, 'def': 31}, {'att': 179, 'def': 20}, 6, 3, 0, 0, 1.4269, 'd', '2015-08-21 22:30:00', 0, 1, 0, 1, 0.1261190000000001]


In [21]:
sevilla_data = []
for row in data['138']['history']:
    sevilla_data.append(list(row.values()))

df = pd.DataFrame(sevilla_data, columns=columns)
df.head(2)

,h_a,xG,xGA,npxG,npxGA,ppda,ppda_allowed,deep,deep_allowed,scored,missed,xpts,result,date,wins,draws,loses,pts,npxGD
0,a,0.856601,0.98272,0.856601,0.98272,"{'att': 179, 'def': 20}","{'att': 146, 'def': 31}",3,6,0,0,1.2439,d,2015-08-21 22:30:00,0,1,0,1,-0.126119
1,h,1.738240,1.24460,1.738240,1.24460,"{'att': 134, 'def': 20}","{'att': 265, 'def': 25}",8,6,0,3,1.7191,l,2015-08-30 22:30:00,0,0,1,0,0.493640


In [22]:
# Getting data for all teams
dataframes = {}
for id, team in teams.items():
    teams_data = []
    for row in data[id]['history']:
        teams_data.append(list(row.values()))
    
    df = pd.DataFrame(teams_data, columns=columns)
    dataframes[team] = df
    print('Added data for {}.'.format(team))

Added data for Malaga.
Added data for Sevilla.
Added data for Deportivo La Coruna.
Added data for Real Sociedad.
Added data for Espanyol.
Added data for Getafe.
Added data for Atletico Madrid.
Added data for Las Palmas.
Added data for Rayo Vallecano.
Added data for Valencia.
Added data for Athletic Club.
Added data for Barcelona.
Added data for Sporting Gijon.
Added data for Real Madrid.
Added data for Levante.
Added data for Celta Vigo.
Added data for Real Betis.
Added data for Villarreal.
Added data for Granada.
Added data for Eibar.


In [23]:
# Sample check of our newly created DataFrame
dataframes['Barcelona'].head(2)

,h_a,xG,xGA,npxG,npxGA,ppda,ppda_allowed,deep,deep_allowed,scored,missed,xpts,result,date,wins,draws,loses,pts,npxGD
0,a,1.76141,0.394999,1.01813,0.394999,"{'att': 126, 'def': 19}","{'att': 446, 'def': 27}",5,5,1,0,2.5392,w,2015-08-23 20:30:00,1,0,0,3,0.623131
1,h,1.92171,0.223011,1.92171,0.223011,"{'att': 90, 'def': 18}","{'att': 274, 'def': 29}",20,3,1,0,2.6354,w,2015-08-29 22:30:00,1,0,0,3,1.698699


In [24]:
for team, df in dataframes.items():
    dataframes[team]['ppda_coef'] = dataframes[team]['ppda'].apply(lambda x: x['att']/x['def'] if x['def'] != 0 else 0)
    dataframes[team]['oppda_coef'] = dataframes[team]['ppda_allowed'].apply(lambda x: x['att']/x['def'] if x['def'] != 0 else 0)

# And check how our new dataframes look based on Sevilla dataframe
dataframes['Sevilla'].head(2)

,h_a,xG,xGA,npxG,npxGA,ppda,ppda_allowed,deep,deep_allowed,scored,...,xpts,result,date,wins,draws,loses,pts,npxGD,ppda_coef,oppda_coef
0,a,0.856601,0.98272,0.856601,0.98272,"{'att': 179, 'def': 20}","{'att': 146, 'def': 31}",3,6,0,...,1.2439,d,2015-08-21 22:30:00,0,1,0,1,-0.126119,8.95,4.709677
1,h,1.738240,1.24460,1.738240,1.24460,"{'att': 134, 'def': 20}","{'att': 265, 'def': 25}",8,6,0,...,1.7191,l,2015-08-30 22:30:00,0,0,1,0,0.493640,6.70,10.600000
